In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.cadena_plos_cb19 import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)

In [2]:
def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
):
    
    load_modules()
    datasets = get_data(
         px_kept=80, final_size=input_size, scale=0.5,
        seed=split_seed
    )

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    ff_1st_block=True,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/ff_1st_block_cb19_data-handle_nan/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
                'handle_nan': True,
            }
        },
        print_model=True,
        handle_nan=True,
    )
    
    return result['stats_best']['stats']['test']['corr_mean'], result['stats_best']['stats']['test']['corr']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 0,
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,
        # 0.000005 gives 0.505823562521005
        # 0.00005 gives 0.5051972572416786
        # 0.0000005 gives 0.5043545818049401
        
        'smoothness_name': '0.000005',
        
        # with smoothness set to 0.00005,
        # scale=0.01 gives 0.505823562521005
        # scale=0.1 gives 0.48046038315427236
        # scale=0.001 gives 0.4979968643549016
        
        # my previous hyperparameters worked the best!!!
        
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 40,
        'n_timesteps': 1,
    }
}

print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular))

# 27629 = 1*2 + 16*1*9*9 + 16*2 + 2*(16*16*3*3+16*16*3*3+4*16*2) + 79*(14*14 + 16 + 1) 

{'final_act', 'pooling', 'bn_output', 'fc'}
['conv0', 'bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv']
num_param 21872
JSONNet(
  (moduledict): ModuleDict(
    (accumulator): RecurrentAccumulator()
    (act0): ReLU()
    (bl_stack): BLConvLayerStack(
      (layer_list): ModuleList(
        (0): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bn_layer_list): ModuleList(
        (0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (capture_list): ModuleList(
        (0): Identity()
        (1): Identity()
      )
      (input_capture): Identity()
      (act_fn): ReLU(inplace=True)
      (pool): Identity()
    )
    (bn

========starting epoch 200==========
200-0, train loss 0.2659338712692261
train loss 0.2659338712692261
val metric {'loss': 0.2790741801261902, 'loss_no_reg': 0.2715302109718323, 'corr': [0.2095256475586634, 0.4282429137713946, 0.4258425141476688, 0.44807032521341733, 0.5941199472548603, 0.4593659084440302, 0.3748915474519689, 0.3979673403050667, 0.2837652140792663, 0.1598456092086356, 0.4266627945076821, 0.14786282798850175, 0.38958034509909234, 0.22283766942371325, 0.45740756743078004, 0.1875101125908149, 0.2519685637046223, 0.08250748959840588, 0.2283792310004717, 0.43916308292611134, 0.3539312523594786, 0.5431262243005237, 0.3173107825185377, 0.36653639638504865, 0.36150096197707365, 0.22581901490037842, 0.4058196679061039, 0.49936115857489655, 0.5919496666135389, 0.4011537099187019, 0.30778132085589244, 0.44928422964192866, 0.5783027160958379, 0.4401186679802091, 0.3763712506322738, 0.4993270273601051, 0.20967819700398743, 0.5317196180395244, 0.46249269709543517, 0.133756548478995

========starting epoch 400==========
400-0, train loss 0.25788116455078125
train loss 0.25788116455078125
val metric {'loss': 0.2705882996320724, 'loss_no_reg': 0.2624967694282532, 'corr': [0.23519289050656889, 0.46090995070158175, 0.43982246752560694, 0.4554751699686586, 0.5983336238952752, 0.4888002907247358, 0.37902688477841595, 0.41101197832334074, 0.28590237744255026, 0.16048336887980558, 0.45648678456553704, 0.1533606155067239, 0.420949692986423, 0.23233290953129412, 0.4953511846809441, 0.19774196975748762, 0.28121808824095845, 0.20289568977579517, 0.24315872538676694, 0.4656264585411439, 0.4227085374358497, 0.5558069936168293, 0.3276340060346472, 0.39971101290674266, 0.44193031407541933, 0.27847958199184436, 0.4403526401793382, 0.5181042577104044, 0.60933538792253, 0.4083106532829649, 0.34535470320567585, 0.44818483709460794, 0.614034983381789, 0.4501071261330846, 0.46152381539877974, 0.5019837626788434, 0.2927947196483506, 0.5290707348453312, 0.4881472845454279, 0.1309287276998

========starting epoch 600==========
600-0, train loss 0.2703830599784851
train loss 0.2703830599784851
val metric {'loss': 0.2657558500766754, 'loss_no_reg': 0.25737476348876953, 'corr': [0.23953608205598143, 0.476204287275635, 0.4373416020887896, 0.45692699107388246, 0.6130158921725753, 0.47650691104561677, 0.38133124322640854, 0.4149884756416604, 0.2827698388696276, 0.15239071027297774, 0.4793013313447355, 0.16442304339641345, 0.5430013995301912, 0.25512222848055277, 0.5000497464908757, 0.20829229695389243, 0.29868052933626954, 0.23270598094279424, 0.24295114189190273, 0.49257578973731103, 0.45143975876397285, 0.5689262798551905, 0.337474232396856, 0.4087864680781283, 0.46476605220966266, 0.3858024377376626, 0.44706900766143925, 0.5127925156151429, 0.6135824687363373, 0.41723169010025324, 0.3562264071238367, 0.44278755890938587, 0.6332698490380606, 0.4576091147764256, 0.4653265681945549, 0.5004676921712837, 0.3159863902932463, 0.5334998271606506, 0.5143266338449095, 0.12583821948505

========starting epoch 800==========
800-0, train loss 0.27251750230789185
train loss 0.27251750230789185
val metric {'loss': 0.2648202657699585, 'loss_no_reg': 0.25648897886276245, 'corr': [0.2391794349477733, 0.46670717140423895, 0.42880228226786016, 0.4659248227843129, 0.6172642303411722, 0.4635012880674599, 0.39054607958385157, 0.4116922505376374, 0.2849679537642384, 0.15315478167852256, 0.4857312580254027, 0.17391600240875899, 0.5576937363672368, 0.2675915333506122, 0.5094026384823631, 0.2015047280934239, 0.31599679086844157, 0.22985605580063478, 0.24278855956455003, 0.4990514964366173, 0.46449551144928053, 0.5758154560138077, 0.340855524849316, 0.40869394152114424, 0.4761013358995283, 0.41188937665696956, 0.4522189453364089, 0.5047932667265588, 0.6098798579635115, 0.416800749758225, 0.3686234671900262, 0.44500637383768354, 0.6363983449297842, 0.4467327097643541, 0.47157182377346385, 0.49707986784982466, 0.3224115133921448, 0.5314727921148606, 0.5255852438346766, 0.130616822134519

...

========starting epoch 200==========
200-0, train loss 0.24156317114830017
train loss 0.24156317114830017
val metric {'loss': 0.262158077955246, 'loss_no_reg': 0.2537291944026947, 'corr': [0.2435200753878741, 0.4792032831966808, 0.4307448524887971, 0.46300791185731643, 0.6160097215601376, 0.4648019012116386, 0.3943787471589412, 0.4104914853464315, 0.28718096221155365, 0.14834733330627883, 0.4959435884031314, 0.18869234118255765, 0.5616404395978523, 0.2691652763612921, 0.5175663498331281, 0.19033674331542824, 0.3111548984506768, 0.23712299812510965, 0.2234347642864564, 0.5119487821807549, 0.4651317929912161, 0.5811851306877824, 0.33989600146793336, 0.4076352901776068, 0.4930455794938972, 0.43195374714490575, 0.462353449121321, 0.5107027377596707, 0.6148240345051919, 0.4177870511786171, 0.3924465066380477, 0.4487569595603439, 0.6377512730476239, 0.4523903273333816, 0.46650396343195016, 0.4978166816946809, 0.32021787657699513, 0.5362235559202119, 0.5358393372131669, 0.12688406950252692, 0

========starting epoch 400==========
400-0, train loss 0.2456347942352295
train loss 0.2456347942352295
val metric {'loss': 0.26202299296855924, 'loss_no_reg': 0.2535983622074127, 'corr': [0.24246929057965147, 0.4804046845838691, 0.42998282760663736, 0.4626627623359123, 0.616938225737247, 0.46393975674138765, 0.39477823290673086, 0.4107187536829411, 0.2888357112085374, 0.1465923516090526, 0.4962024406295701, 0.19098539747382914, 0.5603052930614817, 0.26860889037779373, 0.5181214802812089, 0.19051996125508514, 0.31220740828427923, 0.23469995240230498, 0.22160532624244766, 0.51229050441652, 0.46272078678013573, 0.5823111233735724, 0.3396604365355114, 0.4082109033714573, 0.49377208808234224, 0.4321683557475975, 0.46241258117353046, 0.5102896318046021, 0.6145433208417979, 0.41800020665118764, 0.3936510862346385, 0.44950537165003757, 0.637734195059216, 0.45215201617461803, 0.46725409881380825, 0.49717167852523997, 0.317537391379362, 0.5361337880856567, 0.5359886019155884, 0.1282089144840053

========starting epoch 600==========
600-0, train loss 0.24816802144050598
train loss 0.24816802144050598
val metric {'loss': 0.2624153971672058, 'loss_no_reg': 0.2538919746875763, 'corr': [0.24281471958811235, 0.4793931478647707, 0.43073374766264483, 0.463651453142377, 0.6156786857188836, 0.46473962633346894, 0.39431564682776943, 0.4100242297348789, 0.2911905494187837, 0.14843872940873895, 0.49630335752595073, 0.1911306805324799, 0.5613273381948941, 0.2692848313638617, 0.5176005007568961, 0.19023896370854268, 0.31309620804691146, 0.23394573785373668, 0.22245242896868894, 0.5122213109740577, 0.4635490003965854, 0.5811243013927678, 0.34078145433076923, 0.407698024777416, 0.4945221550826008, 0.4330928484840008, 0.4622540638223276, 0.5101006641145708, 0.615112311000829, 0.4180712633555391, 0.3923716359664017, 0.44923694673278275, 0.6378045862115335, 0.4520802908277346, 0.4671811240031108, 0.49789918848132303, 0.3182496223505459, 0.5359523595612302, 0.5363915253452324, 0.12786140858756653,

========starting epoch 800==========
800-0, train loss 0.23971781134605408
train loss 0.23971781134605408
val metric {'loss': 0.2624888360500336, 'loss_no_reg': 0.2539505362510681, 'corr': [0.24306879486483426, 0.479757693088746, 0.43010784584396566, 0.4625567581538528, 0.6160918543611399, 0.46456912433097913, 0.39504617836083955, 0.4103710407655855, 0.2945151966115523, 0.1471193721246143, 0.49622899902039624, 0.19273525290485674, 0.5622996969316617, 0.26805883674518455, 0.5172717494600705, 0.18909148488111208, 0.3112284179986168, 0.23466370106910195, 0.2190168965616422, 0.5123470072077252, 0.4628732063162013, 0.5822471223103303, 0.3411265012214359, 0.40715166408394743, 0.49439455139981303, 0.4319242653532673, 0.46142364192176766, 0.5103124632414057, 0.614368504653668, 0.417515719815754, 0.39384036682183915, 0.4486094109367635, 0.6369173118953441, 0.45252661451186077, 0.46690624362687794, 0.49788599710620346, 0.315747653901728, 0.5357700239159012, 0.5355846987540315, 0.1274395998768440

(0.505823562521005, [0.22290867814140142, 0.4708074534173289, 0.4748005428071667, 0.4726406797165408, 0.6261681532403353, 0.47778410642306074, 0.355276401771913, 0.4059499988799383, 0.27306687854409467, 0.2060898428331426, 0.5066163009016792, 0.21927353933146942, 0.5601374651206583, 0.2686960626493656, 0.5464271834695443, 0.2654781651778954, 0.32077724062824814, 0.23326450810839897, 0.3647180023372379, 0.47421547330506253, 0.3785651481083519, 0.5456766079217238, 0.3364432552323184, 0.4076604394937908, 0.4603158349474313, 0.3521687998817147, 0.4759313387428043, 0.5159867024433585, 0.6517636553453058, 0.44227082061214457, 0.42572542630461446, 0.46016411728227063, 0.6354862858793571, 0.48589746898023056, 0.4898000842666996, 0.5060820441937934, 0.3344468595601764, 0.5575963223930174, 0.565608854549304, 0.14956376027860313, 0.3688827327773246, 0.5157210848584867, 0.37949105748215006, 0.38131425254857293, 0.29023178616260425, 0.4092873045632762, 0.6546056913526376, 0.5105307686644647, 0.4648